In [1]:
import pandas as pd
import numpy as np

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error

import warnings

warnings.filterwarnings('ignore')

C:\Users\X1Carbon\.conda\envs\mlproject\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
data = pd.read_csv('pretty_beer_preprocessing.csv', encoding='utf-8', index_col=0)

In [3]:
tmp = data.copy()

# 10개이상 리뷰를 남긴 유저, 맥주만 남기기

In [4]:
# n개 이상의 데이터 리뷰를 남긴 유저, 맥주를 걸러내는 함수
def preprocessing(data, n):
    min_id = data['ID'].value_counts() >= n
    min_id = min_id[min_id].index.to_list()
    data = data[data['ID'].isin(min_id)]

    min_beer = data['beer_name'].value_counts() >= n
    min_beer = min_beer[min_beer].index.to_list()
    data = data[data['beer_name'].isin(min_beer)]
    
    return data

temp=tmp.copy()

In [5]:
# 10번 반복합니다.
for i in range(1,10):
    temp = preprocessing(temp, 10)
    print(temp.shape)

temp.to_csv('beer_review_over10.csv', encoding='utf-8')

(49160, 10)
(49160, 10)
(49160, 10)
(49160, 10)
(49160, 10)
(49160, 10)
(49160, 10)
(49160, 10)
(49160, 10)


# 맥주간 유사도 기반 추천
맥주간 유사도를 통해 비슷한 맥주 추천

코사인 유사도를 사용

아이디, 맥주, 평점으로 매트릭스 구성

## 유저 - 아이디 매트릭스 구성

In [6]:
data = pd.read_csv('beer_review_over10.csv', encoding='utf-8', index_col=0)

ratings = data.copy()

# 피벗 테이블을 이용해 유저-아이디 매트릭스 구성
ratings_matrix = ratings.pivot_table('grade', index='ID', columns='beer_name')
ratings_matrix.head(3)

# fillna함수를 이용해 Nan처리
ratings_matrix = ratings_matrix.fillna(0)
ratings_matrix

beer_name,Asahi Super Dry,Asahi Super Dry Black,Bavaria 8.6 (Original),Bavaria Pilsener / Premium Beer,Beck's,Berliner Kindl Jubiläums Pilsener Premium,Bintang Pilsener,Blue Moon Belgian White,Brooklyn Special Effects (Hoppy Amber/Lager),Budweiser,...,Stephans Bräu Premium Pilsener,Suntory The Premium Malt's,Terra,Tiger Beer,Tiger Radler Lemon,Tsingtao,Tsingtao Wheat Beer,Victoria Bitter,Volfas Engelman Grünberger Hefeweizen,Warsteiner Premium Verum
ID,,,,,,,,,,,,,,,,,,,,,
007lund(92),0.0,0.0,0.0,0.0,2.6,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,2.5,0.0,0.0,0.0,4.1
"00cobraR(1,103)",0.0,0.0,0.0,0.0,2.3,0.0,0.0,1.7,0.0,1.4,...,0.0,0.0,0.0,0.0,0.0,2.1,0.0,0.0,0.0,0.0
1010(309),2.4,0.0,0.0,0.0,2.0,0.0,0.0,2.5,0.0,0.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12HundredBaud(262),2.3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16erBlech(560),0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,3.0,0.0,2.0,0.0,2.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"zenkis(1,051)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ziobarba2000(120),0.0,0.0,0.0,1.6,1.6,0.0,0.0,2.7,0.0,2.4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.4
zombywoof(537),0.0,0.0,0.0,1.5,1.8,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.5


In [11]:
# 유저수 : 2668, 맥주 수 : 74

In [ ]:
# 유저 X 맥주 매트릭스에서 맥주 X 유저 매트릭스로 변환

## 유사도 계산을 위해 트랜스포즈

In [7]:
# 유사도 계산을 위해 트랜스포즈
ratings_matrix_T = ratings_matrix.transpose()
ratings_matrix_T

ID,007lund(92),"00cobraR(1,103)",1010(309),12HundredBaud(262),16erBlech(560),"17Beer(1,728)","17thfloor(2,444)","17wojownik(1,452)",184601(206),"1FastSTi(3,361)",...,"zathrus13(1,776)",zborgerd(187),zdet444dd(440),"zdk(1,917)","zebracakes(1,339)","zenkis(1,051)",ziobarba2000(120),zombywoof(537),"zvikar(12,470)","zvsn(7,892)"
beer_name,,,,,,,,,,,,,,,,,,,,,
Asahi Super Dry,0.0,0.0,2.4,2.3,0.0,3.4,2.0,0.0,2.4,0.0,...,0.0,1.8,1.8,1.8,2.7,0.0,0.0,0.0,1.2,0.0
Asahi Super Dry Black,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.5,0.0
Bavaria 8.6 (Original),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
Bavaria Pilsener / Premium Beer,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7,0.0,...,0.0,0.0,0.0,3.0,0.0,0.0,1.6,1.5,0.0,0.0
Beck's,2.6,2.3,2.0,0.0,0.0,0.0,2.8,2.9,2.8,2.5,...,0.0,0.0,0.0,1.8,2.1,0.0,1.6,1.8,2.1,1.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Tsingtao,2.5,2.1,0.0,0.0,3.0,2.8,2.5,2.1,0.9,0.0,...,2.2,2.3,2.0,1.8,1.7,0.0,0.0,0.0,1.7,0.0
Tsingtao Wheat Beer,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Victoria Bitter,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.4,0.0


## 아이템 - 유저 매트릭스로부터 코사인 유사도 구하기

In [9]:
# 아이템-유저 매트릭스로부터 코사인 유사도 구하기
item_sim = cosine_similarity(ratings_matrix_T, ratings_matrix_T)

# cosine_similarity()로 반환된 넘파이 행렬에 맥주명을 매핑해 DataFrame으로 변환
item_sim_df = pd.DataFrame(data=item_sim, index=ratings_matrix.columns,
                          columns=ratings_matrix.columns)

print(item_sim_df.shape)
item_sim_df.head(3)

(74, 74)


beer_name,Asahi Super Dry,Asahi Super Dry Black,Bavaria 8.6 (Original),Bavaria Pilsener / Premium Beer,Beck's,Berliner Kindl Jubiläums Pilsener Premium,Bintang Pilsener,Blue Moon Belgian White,Brooklyn Special Effects (Hoppy Amber/Lager),Budweiser,...,Stephans Bräu Premium Pilsener,Suntory The Premium Malt's,Terra,Tiger Beer,Tiger Radler Lemon,Tsingtao,Tsingtao Wheat Beer,Victoria Bitter,Volfas Engelman Grünberger Hefeweizen,Warsteiner Premium Verum
beer_name,,,,,,,,,,,,,,,,,,,,,
Asahi Super Dry,1.000000,0.199375,0.321553,0.477022,0.600823,0.250984,0.407218,0.546751,0.180407,0.602621,...,0.139687,0.258566,0.101746,0.611003,0.093164,0.641571,0.111463,0.414656,0.043188,0.571494
Asahi Super Dry Black,0.199375,1.000000,0.115623,0.157963,0.168876,0.099513,0.174373,0.159385,0.079052,0.159277,...,0.053182,0.359579,0.077141,0.190185,0.166640,0.148777,0.053647,0.189259,0.063365,0.165546
Bavaria 8.6 (Original),0.321553,0.115623,1.000000,0.388215,0.316759,0.204017,0.281060,0.240981,0.148921,0.297185,...,0.176558,0.097627,0.028424,0.313056,0.080142,0.298254,0.093221,0.264760,0.065669,0.313087


## 74개의 맥주 종류확인

In [10]:
ratings_matrix.columns

Index(['Asahi Super Dry', 'Asahi Super Dry Black', 'Bavaria 8.6 (Original)',
       'Bavaria Pilsener / Premium Beer', 'Beck's',
       'Berliner Kindl Jubiläums Pilsener Premium', 'Bintang Pilsener',
       'Blue Moon Belgian White',
       'Brooklyn Special Effects (Hoppy Amber/Lager)', 'Budweiser', 'Cafri',
       'Carlsberg Pilsner', 'Cass Fresh', 'Cass Light',
       'Castlemaine XXXX Gold Lager', 'Desperados',
       'Edelweiss Weissbier Snowfresh', 'Egger Grapefruit Radler',
       'Egger Märzenbier', 'Egger Zwickl', 'Eichbaum Apostel Bräu',
       'Erdinger Weissbier', 'Erdinger Weissbier Dunkel',
       'Franziskaner Hefe-Weissbier / Weissbier Naturtrub',
       'Gambrinus Originál 10° (Desítka)', 'Grolsch Premium Lager',
       'Guinness Draught', 'Guinness Original 4.2% (Ireland/UK)',
       'Harbin Beer', 'Heineken', 'Heineken Dark Lager', 'Heineken Silver',
       'Hite Beer', 'Hite Black Beer Stout', 'Hite D (Dry Finish)',
       'Hite Extra Cold', 'Hite Prime Max', 'Hoeg

## 특정 맥주와 유사도가 높은 맥주 5개 추출해보기

In [11]:
# 코젤 맥주와 유사도가 높은 맥주 5개만 추출하기
item_sim_df['Kozel Černý (Dark) 10°'].sort_values(ascending=False)[:5]

beer_name
Kozel Černý (Dark) 10°                               1.000000
Leffe Brune / Bruin / Brown                          0.548356
Gambrinus Originál 10° (Desítka)                     0.509911
Erdinger Weissbier                                   0.506796
Franziskaner Hefe-Weissbier / Weissbier Naturtrub    0.501577
Name: Kozel Černý (Dark) 10°, dtype: float64

In [12]:
# 호가든 맥주와 유사도가 높은 맥주 5개만 추출하기
item_sim_df['Hoegaarden'].sort_values(ascending=False)[:5]

beer_name
Hoegaarden                                           1.000000
Guinness Draught                                     0.816028
Franziskaner Hefe-Weissbier / Weissbier Naturtrub    0.795210
Paulaner Hefe-Weissbier                              0.791244
Pilsner Urquell                                      0.785264
Name: Hoegaarden, dtype: float64

# 개인의 평점이 반영된 맥주 추천

In [13]:
# ratings_arr.dot(item_sim_arr)는 평점 * 맥주 유사도
# ratings_arr는 사용자 u의 아이템 i와 가장 유사도가 높은 Top_N개 아이템에 대한 실제 평점 벡터
# item_sim_arr는 아이템 i와 가장 유사도가 높은 Top_N개 아이템의 유사도 벡터
def predict_rating(ratings_arr, item_sim_arr):
    ratings_pred = ratings_arr.dot(item_sim_arr) / np.array([np.abs(item_sim_arr).sum(axis=1)])
    return ratings_pred

# 개인화된 예측 평점 구하기
# 평점 value와 유사도 value만 뽑아서 대입
ratings_pred = predict_rating(ratings_matrix.values, item_sim_df.values)
ratings_pred_matrix = pd.DataFrame(data=ratings_pred, index=ratings_matrix.index,
                                  columns = ratings_matrix.columns)

# 개인별로 계산된 예측 평점
ratings_pred_matrix

beer_name,Asahi Super Dry,Asahi Super Dry Black,Bavaria 8.6 (Original),Bavaria Pilsener / Premium Beer,Beck's,Berliner Kindl Jubiläums Pilsener Premium,Bintang Pilsener,Blue Moon Belgian White,Brooklyn Special Effects (Hoppy Amber/Lager),Budweiser,...,Stephans Bräu Premium Pilsener,Suntory The Premium Malt's,Terra,Tiger Beer,Tiger Radler Lemon,Tsingtao,Tsingtao Wheat Beer,Victoria Bitter,Volfas Engelman Grünberger Hefeweizen,Warsteiner Premium Verum
ID,,,,,,,,,,,,,,,,,,,,,
007lund(92),0.762899,0.427904,0.591772,0.690894,0.854707,0.543761,0.591330,0.817920,0.525504,0.871025,...,0.404660,0.489223,0.410125,0.731454,0.329210,0.836694,0.342281,0.620241,0.310537,0.878096
"00cobraR(1,103)",0.808846,0.459918,0.616883,0.733083,0.897673,0.584468,0.622311,0.916845,0.582492,0.912998,...,0.440172,0.517832,0.445740,0.775574,0.353835,0.882187,0.383030,0.648708,0.356938,0.885695
1010(309),0.692948,0.391799,0.498571,0.588642,0.722033,0.472364,0.507622,0.777656,0.492805,0.731475,...,0.373701,0.434375,0.366686,0.634705,0.293762,0.694085,0.328462,0.524703,0.282603,0.713711
12HundredBaud(262),0.992202,0.636488,0.853332,0.928124,1.008342,0.813896,0.832285,1.014181,0.851458,1.013884,...,0.637473,0.684600,0.604161,0.953084,0.509303,0.994979,0.555208,0.849610,0.501458,1.024622
16erBlech(560),0.987830,0.686271,0.883865,0.949665,1.015193,1.063681,0.886196,1.028856,0.901371,1.016708,...,0.752083,0.753218,0.635299,0.988176,0.497950,1.049936,0.594510,0.955986,0.664962,1.066504
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"zenkis(1,051)",0.535990,0.349724,0.479299,0.520241,0.567479,0.496584,0.463944,0.570665,0.496520,0.569317,...,0.425646,0.366459,0.314953,0.534233,0.229939,0.553893,0.343277,0.473747,0.414337,0.576577
ziobarba2000(120),0.598044,0.385986,0.514996,0.600285,0.651825,0.481726,0.500610,0.687964,0.497421,0.681818,...,0.385191,0.430169,0.363546,0.588768,0.263087,0.623259,0.319090,0.520897,0.312410,0.672473
zombywoof(537),0.957475,0.570246,0.767533,0.924811,1.060036,0.740919,0.754167,1.039775,0.729581,1.067509,...,0.557369,0.623464,0.525666,0.931244,0.449115,1.006848,0.470553,0.792938,0.457199,1.084269


## 평균제곱오차 (MSE, Mean Squared Error)

머신러닝뿐만 아니라 영상처리 영역에서도 자주 사용되는 추측값에 대한 정확성을 측정하는 방법

추정한 값에 대한 정확도를 측정하는 방법

In [14]:
# 우리가 예측한 평점과 실제 평점간의 차이를 MSE(평균제곱오차)로 계산
def get_mse(pred, actual):
    # 평점이 있는 실제 맥주만 추출
    pred = pred[actual.nonzero()].flatten()
    actual = actual[actual.nonzero()].flatten()
    return mean_squared_error(pred, actual)

print('아이템 기반 모든 최근접 이웃 MSE: ', 
      get_mse(ratings_pred, ratings_matrix.values))

아이템 기반 모든 최근접 이웃 MSE:  2.689477375269283


# Top-N 기반 추천

MSE 값을 더 낮추기 위해 특정 맥주와 가장 비슷한 N개의 맥주들만 유사도 계산에 사용

MSE가 0에 가까울 수록 추측한 값이 원본에 가까움 (예측값과 실제값 차이의 면적의 평균)

위의 방식은 모든 맥주들의 유사도 백터를 사용했기 때문에 MSE 값이 다소 낮음

In [15]:
# 3개의 col까지만. 3개의 맥주에 대해서 유사도가 큰 5개 선택
top_n_items = [np.argsort(item_sim_df.values[:,3])[:-5:-1]]
top_n_items

[array([ 3, 25, 37,  4], dtype=int64)]

In [16]:
# 따라서 가장 비슷한 유사도를 가지는 맥주만 유사도 벡터로 사용
# 특정 맥주와 비슷한 유사도를 가지는 맥주 Top_N에 대해서만 적용 -> 시간오래걸림

def predict_rating_topsim(ratings_arr, item_sim_arr, n=20):
    # 사용자-아이템 평점 행렬 크기만큼 0으로 채운 예측 행렬 초기화
    pred = np.zeros(ratings_arr.shape)

    # 사용자-아이템 평점 행렬의 맥주 개수만큼 루프
    for col in range(ratings_arr.shape[1]):
        # 유사도 행렬에서 유사도가 큰 순으로 n개의 데이터 행렬의 인덱스 반환
        top_n_items = [np.argsort(item_sim_arr[:, col])[:-n-1:-1]]
        # 개인화된 예측 평점 계산 : 각 col 맥주별(1개), 2496 사용자들의 예측평점
        for row in range(ratings_arr.shape[0]):
            pred[row, col] = item_sim_arr[col,:][top_n_items].dot(
            ratings_arr[row, :][top_n_items].T)
            pred[row, col] /= np.sum(item_sim_arr[col,:][top_n_items])
    return pred

ratings_pred = predict_rating_topsim(ratings_matrix.values, item_sim_df.values, n=10)
print('아이템 기반 최근접 TOP-N 이웃 MSE: ', 
      get_mse(ratings_pred, ratings_matrix.values))

# 계산된 예측 평점 데이터는 DataFrame으로 재생성
ratings_pred_matrix = pd.DataFrame(data=ratings_pred, index=ratings_matrix.index,
                                  columns=ratings_matrix.columns)

ratings_pred_matrix

아이템 기반 최근접 TOP-N 이웃 MSE:  0.8150278396435262


beer_name,Asahi Super Dry,Asahi Super Dry Black,Bavaria 8.6 (Original),Bavaria Pilsener / Premium Beer,Beck's,Berliner Kindl Jubiläums Pilsener Premium,Bintang Pilsener,Blue Moon Belgian White,Brooklyn Special Effects (Hoppy Amber/Lager),Budweiser,...,Stephans Bräu Premium Pilsener,Suntory The Premium Malt's,Terra,Tiger Beer,Tiger Radler Lemon,Tsingtao,Tsingtao Wheat Beer,Victoria Bitter,Volfas Engelman Grünberger Hefeweizen,Warsteiner Premium Verum
ID,,,,,,,,,,,,,,,,,,,,,
007lund(92),1.352318,0.000000,0.000000,1.052577,2.380067,0.000000,0.375773,1.966653,0.000000,2.234478,...,0.000000,0.000000,0.000000,1.349192,0.000000,2.040538,0.000000,0.445394,0.000000,2.263783
"00cobraR(1,103)",1.529471,0.000000,0.237147,1.215743,2.423075,0.000000,0.374388,2.442847,0.279260,2.203268,...,0.000000,0.000000,0.000000,1.612284,0.000000,2.556362,0.000000,0.630849,0.000000,2.558584
1010(309),1.528673,0.000000,0.188076,0.884352,1.770189,0.000000,0.459467,2.307754,0.323353,1.766349,...,0.000000,0.173193,0.000000,1.648261,0.000000,2.265449,0.000000,0.674608,0.000000,2.064220
12HundredBaud(262),1.628998,0.000000,1.553372,1.857516,1.677827,0.825194,0.671049,1.803168,1.172819,1.408505,...,0.152743,0.165976,0.000000,1.870168,0.000000,1.985591,0.207596,1.161579,0.000000,2.082403
16erBlech(560),1.521112,0.000000,0.943109,1.535238,1.272960,1.971541,0.936805,1.621694,0.920943,1.317205,...,0.371652,0.146566,0.000000,1.606438,0.000000,1.806178,0.000000,1.895630,0.144915,1.732063
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"zenkis(1,051)",0.933789,0.000000,0.490101,0.957395,1.220589,0.679319,0.195798,1.215066,0.610265,1.041097,...,0.341791,0.000000,0.000000,0.931274,0.000000,1.215605,0.172777,0.522774,0.246355,1.234084
ziobarba2000(120),0.631856,0.000000,0.340854,0.930073,1.453832,0.232630,0.130377,1.637442,0.175738,1.408078,...,0.149383,0.000000,0.000000,0.783905,0.000000,0.937325,0.000000,0.250630,0.000000,1.287381
zombywoof(537),1.822165,0.125494,0.592608,1.971842,2.773209,0.234966,0.415899,2.647746,0.443994,2.561476,...,0.000000,0.000000,0.000000,1.850049,0.093578,2.480149,0.000000,0.809459,0.000000,3.102005


MSE 값이 감소, 예측 행렬 값도 변했음

# 사용자에게 맥주 추천하기

## 사용자 아이디와 평점이 주어진 상황에서 맥주 10개 추천

사용자 아이디 입력

In [17]:
# 행렬에 존재하는 user 정보 사용해서 추천해보기

# username='snoworsummer(8,581)'
username='16erBlech(560)'

# 특정 유저 대상으로 맥주추천
user_rating_id = ratings_matrix.loc[username, :]
# 유저가 먹었던 맥주들 출력
user_rating_id[user_rating_id > 0].sort_values(ascending=False)[:10]

beer_name
Leffe Brune / Bruin / Brown                          4.5
Hoegaarden                                           4.5
Kozel Černý (Dark) 10°                               4.0
Paulaner Hefe-Weissbier                              4.0
Franziskaner Hefe-Weissbier / Weissbier Naturtrub    3.5
Tsingtao                                             3.0
Schöfferhofer Grapefruit                             3.0
Berliner Kindl Jubiläums Pilsener Premium            3.0
Kirin Ichiban                                        3.0
Guinness Original 4.2% (Ireland/UK)                  3.0
Name: 16erBlech(560), dtype: float64

## 사용자가 먹어본 맥주 제외시키기

In [18]:
# 사용자가 먹어본 맥주 제외시키기

# 사용자가 안 먹어본 맥주를 추천하자.
def get_not_tried_beer(ratings_matrix, userId):
    # userId로 입력받은 사용자의 모든 맥주 정보를 추출해 Series로 반환
    # 반환된 user_rating은 beer_name을 인덱스로 가지는 Series 객체
    user_rating = ratings_matrix.loc[userId, :]

    # user_rating이 0보다 크면 기존에 마셔본 맥주.
    # 대상 인덱스를 추출해 list 객체로 만듦
    tried = user_rating[user_rating>0].index.tolist()

    # 모든 맥주명을 list 객체로 만듦
    beer_list = ratings_matrix.columns.tolist()

    # list comprehension으로 tried에 해당하는 맥주는 beer_list에서 제외
    not_tried = [beer for beer in beer_list if beer not in tried]

    return not_tried

# 예측 평점 DataFrame에서 사용자 id 인덱스와 not_tried로 들어온 맥주명 추출 후
# 가장 예측 평점이 높은 순으로 정렬
def recomm_beer_by_userid(pred_df, userId, not_tried, top_n):
    recomm_beer = pred_df.loc[userId, not_tried].sort_values(ascending=False)[:top_n]
    return recomm_beer

# 유저가 먹지 않은 맥주이름 추출
not_tried = get_not_tried_beer(ratings_matrix, username)
len(not_tried)

57

## 아이템 기반의 최근접 이웃 CF로 맥주 추천

In [19]:
# top_n과 비슷한 맥주만 추천에 사용
ratings_pred = predict_rating_topsim(ratings_matrix.values, item_sim_df.values, n=5)

# 계산된 예측 평점 데이터는 DataFrame으로 재생성
ratings_pred_matrix = pd.DataFrame(data=ratings_pred, index=ratings_matrix.index,
                                   columns=ratings_matrix.columns)

# 유저가 먹지 않은 맥주이름 추출
not_tried = get_not_tried_beer(ratings_matrix, username)

# 아이템 기반의 최근접 이웃 CF로 맥주 추천
recomm_beer = recomm_beer_by_userid(ratings_pred_matrix, username, not_tried, top_n=10)
recomm_beer = pd.DataFrame(data=recomm_beer.values, index=recomm_beer.index,
                           columns=['예측평점'])
recomm_beer

,예측평점
beer_name,
Erdinger Weissbier Dunkel,2.740646
Krombacher Weizen,2.524082
Pilsner Urquell,2.246264
Kronenbourg 1664 Blanc,1.988996
Blue Moon Belgian White,1.899661
Tiger Beer,1.876493
Asahi Super Dry,1.802769
Gambrinus Originál 10° (Desítka),1.413425
Hoegaarden Rosée,1.382601
